# Aligning a trajectory to a reference

We use `align.AlignTraj` to align a trajectory to a frame in a reference trajectory and write it to a file.

**Last updated:** December 2022

**Minimum version of MDAnalysis:** 2.0.0

**Packages required:**
    
* MDAnalysis (<a data-cite="michaud-agrawal_mdanalysis_2011" href="https://doi.org/10.1002/jcc.21787">Michaud-Agrawal *et al.*, 2011</a>, <a data-cite="gowers_mdanalysis_2016" href="https://doi.org/10.25080/Majora-629e541a-00e">Gowers *et al.*, 2016</a>)
* MDAnalysisTests
   
**Optional packages for molecular visualisation:**
    
* [nglview](http://nglviewer.org/nglview/latest) (<a data-cite="nguyen_nglviewinteractive_2018" href="https://doi.org/10.1093/bioinformatics/btx789">Nguyen *et al.*, 2018</a>)

Throughout this tutorial we will include cells for visualising Universes with the [NGLView](http://nglviewer.org/nglview/latest/api.html) library. However, these will be commented out, and we will show the expected images generated instead of the interactive widgets.


**See also**

* [Aligning a trajectory to a frame from itself](aligning_trajectory_to_frame.ipynb)
* [Aligning a structure to another](aligning_structure_to_another.ipynb)
* [RMSD](rmsd.ipynb)

<div class="alert alert-info">
    
**Note**

MDAnalysis implements RMSD calculation using the fast QCP algorithm (<a data-cite="theobald_rapid_2005" href="https://doi.org/10.1107/S0108767305015266">Theobald, 2005</a>) and a rotation matrix *R* that minimises the RMSD (<a data-cite="liu_fast_2009" href="https://doi.org/10.1002/jcc.21439">Liu *et al.*, 2009</a>). Please cite (<a data-cite="theobald_rapid_2005" href="https://doi.org/10.1107/S0108767305015266">Theobald, 2005</a>) and (<a data-cite="liu_fast_2009" href="https://doi.org/10.1002/jcc.21439">Liu *et al.*, 2009</a>) when using the ``MDAnalysis.analysis.align`` module in published work.

</div>

In [1]:
import MDAnalysis as mda
from MDAnalysis.analysis import align
from MDAnalysis.tests.datafiles import CRD, PSF, DCD, DCD2
# import nglview as nv

import warnings
# suppress some MDAnalysis warnings when writing PDB files
warnings.filterwarnings('ignore')

## Loading files

The test files we will be working with here are trajectories of a adenylate kinase (AdK), a phosophotransferase enzyme. (<a data-cite="beckstein_zipping_2009" href="https://doi.org/10.1016/j.jmb.2009.09.009">Beckstein *et al.*, 2009</a>) The trajectories sample a transition from a closed to an open conformation. 

In [2]:
adk_open = mda.Universe(CRD, DCD2)
adk_closed = mda.Universe(PSF, DCD)

Currently, the proteins are not aligned to each other. The difference becomes obvious when the closed conformation is compared to the open. Below, we set `adk_open` to the last frame and see the relative positions of each protein in a merged Universe.

In [3]:
adk_open.trajectory[-1] # last frame
merged = mda.Merge(adk_open.atoms, adk_closed.atoms)

In [4]:
# merged_view = nv.show_mdanalysis(merged)
# merged_view

<center>
<div style='width: 800px'>
    
![merged_view](images/merged_view.png)
    
</div>
</center>

## Aligning a trajectory with AlignTraj

While [align.alignto](https://docs.mdanalysis.org/stable/documentation_pages/analysis/align.html#MDAnalysis.analysis.align.alignto) aligns structures, or a frame of a trajectory, `align.AlignTraj` ([API docs](https://docs.mdanalysis.org/stable/documentation_pages/analysis/align.html#MDAnalysis.analysis.align.AlignTraj)) efficiently aligns an entire trajectory to a reference. Unlike most other analysis modules, `AlignTraj` allows you to write the output of the analysis to a file. This is because when `Universe`s are created by loading from a file, changes to frame-by-frame (dynamic) information [do not persist](https://userguide.mdanalysis.org/stable/trajectories/trajectories.html) when the frame is changed. If the trajectory is not written to a file, or pulled into memory (below), `AlignTraj` will have no effect.

In [5]:
align.AlignTraj(adk_closed,  # trajectory to align
                adk_open,  # reference
                select='name CA',  # selection of atoms to align
                filename='aligned.dcd',  # file to write the trajectory to
                match_atoms=True,  # whether to match atoms based on mass
               ).run()
# merge adk_closed and adk_open into the same universe
merged1 = mda.Merge(adk_closed.atoms, adk_open.atoms)

In [6]:
# merged1_view = nv.show_mdanalysis(merged1)
# merged1_view

<center>
<div style='width: 800px'>
    
![merged1_view](images/aligning_trajectory-merged1_view.png)
    
</div>
</center>

As you can see, the `adk_closed` and `adk_open` trajectories still look the same. However, when we load our aligned trajectory from `aligned.dcd`, we can see them superposed:

In [7]:
aligned = mda.Universe(PSF, 'aligned.dcd')
aligned.segments.segids = ['Aligned']  # rename our segments
adk_open.segments.segids = ['Open']  # so they're coloured differently
merged2 = mda.Merge(aligned.atoms, adk_open.atoms)

In [8]:
# merged2_view = nv.show_mdanalysis(merged2)
# merged2_view

<center>
<div style='width: 800px'>
    
![merged2_view](images/aligning_trajectory-merged2_view.png)
    
</div>
</center>

If you don't want to write a file, you can also choose to load the entire trajectory into memory. (This is not always feasible depending on how large your trajectory is, and how much memory your device has, in which case it is much more efficient to write an aligned trajectory to a file as above). You can accomplish this in one of two ways:

1. Load the trajectory into memory in the first place
```python
adk_closed = mda.Universe(PSF, DCD, in_memory=True)
```

2. Select `in_memory=True` during `AlignTraj` (below)

In [9]:
align.AlignTraj(adk_closed,  # trajectory to align
                adk_open,  # reference
                select='name CA',  # selection of atoms to align
                filename='aligned.dcd',  # file to write the trajectory to
                match_atoms=True,  # whether to match atoms based on mass
                in_memory=True
               ).run()
# merge adk_closed and adk_open into the same universe
merged3 = mda.Merge(adk_closed.atoms, adk_open.atoms)

## Copying coordinates into a new Universe

`MDAnalysis.Merge` does not automatically load coordinates for a trajectory. We can do this ourselves. Below, we copy the coordinates of the 98 frames in the `aligned` universe.

In [10]:
from MDAnalysis.analysis.base import AnalysisFromFunction
import numpy as np
from MDAnalysis.coordinates.memory import MemoryReader

def copy_coords(ag):
    return ag.positions.copy()

aligned_coords = AnalysisFromFunction(copy_coords, 
                                      aligned.atoms).run().results

print(aligned_coords['timeseries'].shape)

(98, 3341, 3)


To contrast, we will keep the open conformation static.

In [11]:
adk_coords = adk_open.atoms.positions.copy()
adk_coords.shape

(3341, 3)

Because there are 98 frames of the `aligned` Universe, we copy the coordinates of the `adk_open` positions and stack them.

In [12]:
adk_traj_coords = np.stack([adk_coords] * 98)
adk_traj_coords.shape

(98, 3341, 3)

We join `aligned_coords` and `adk_traj_coords` on the second axis with `np.hstack` and load the coordinates into memory into the `merged2` Universe.

In [13]:
merged_coords = np.hstack([aligned_coords['timeseries'], 
                           adk_traj_coords])
merged2.load_new(merged_coords, format=MemoryReader)

<Universe with 6682 atoms>

In [14]:
# m2_view = nv.show_mdanalysis(merged2)
# m2_view

Online notebooks do not show the molecule trajectory, but here you can use `nglview.contrib.movie.MovieMaker` to make a gif of the trajectory. This requires you to install `moviepy`.

In [15]:
# from nglview.contrib.movie import MovieMaker
# movie = MovieMaker(
#     m2_view,
#     step=4,  # only render every 4th frame
#     output='merged.gif',
#     render_params={"factor": 3},  # set to 4 for higher quality
# )
# movie.make()

<center>
<div style='width: 800px'>
    
![merged](merged.gif)
    
</div>
</center>

## Writing trajectories to a file

Finally, we can also save this new trajectory to a file. 

In [16]:
with mda.Writer('aligned.xyz', merged2.atoms.n_atoms) as w:
    for ts in merged2.trajectory:
        w.write(merged2.atoms)

## References

[1] Oliver Beckstein, Elizabeth&nbsp;J. Denning, Juan&nbsp;R. Perilla, and Thomas&nbsp;B. Woolf.
Zipping and <span class="bibtex-protected">Unzipping</span> of <span class="bibtex-protected">Adenylate</span> <span class="bibtex-protected">Kinase</span>: <span class="bibtex-protected">Atomistic</span> <span class="bibtex-protected">Insights</span> into the <span class="bibtex-protected">Ensemble</span> of <span class="bibtex-protected">Open</span>↔<span class="bibtex-protected">Closed</span> <span class="bibtex-protected">Transitions</span>.
<em>Journal of Molecular Biology</em>, 394(1):160–176, November 2009.
00107.
URL: <a href="https://linkinghub.elsevier.com/retrieve/pii/S0022283609011164">https://linkinghub.elsevier.com/retrieve/pii/S0022283609011164</a>, <a href="https://doi.org/10.1016/j.jmb.2009.09.009">doi:10.1016/j.jmb.2009.09.009</a>.

[2] Richard&nbsp;J. Gowers, Max Linke, Jonathan Barnoud, Tyler J.&nbsp;E. Reddy, Manuel&nbsp;N. Melo, Sean&nbsp;L. Seyler, Jan Domański, David&nbsp;L. Dotson, Sébastien Buchoux, Ian&nbsp;M. Kenney, and Oliver Beckstein.
<span class="bibtex-protected">MDAnalysis</span>: <span class="bibtex-protected">A</span> <span class="bibtex-protected">Python</span> <span class="bibtex-protected">Package</span> for the <span class="bibtex-protected">Rapid</span> <span class="bibtex-protected">Analysis</span> of <span class="bibtex-protected">Molecular</span> <span class="bibtex-protected">Dynamics</span> <span class="bibtex-protected">Simulations</span>.
<em>Proceedings of the 15th Python in Science Conference</em>, pages 98–105, 2016.
00152.
URL: <a href="https://conference.scipy.org/proceedings/scipy2016/oliver_beckstein.html">https://conference.scipy.org/proceedings/scipy2016/oliver_beckstein.html</a>, <a href="https://doi.org/10.25080/Majora-629e541a-00e">doi:10.25080/Majora-629e541a-00e</a>.

[3] Pu&nbsp;Liu, Dimitris&nbsp;K. Agrafiotis, and Douglas&nbsp;L. Theobald.
Fast determination of the optimal rotational matrix for macromolecular superpositions.
<em>Journal of Computational Chemistry</em>, pages n/a–n/a, 2009.
URL: <a href="http://doi.wiley.com/10.1002/jcc.21439">http://doi.wiley.com/10.1002/jcc.21439</a>, <a href="https://doi.org/10.1002/jcc.21439">doi:10.1002/jcc.21439</a>.

[4] Naveen Michaud-Agrawal, Elizabeth&nbsp;J. Denning, Thomas&nbsp;B. Woolf, and Oliver Beckstein.
<span class="bibtex-protected">MDAnalysis</span>: <span class="bibtex-protected">A</span> toolkit for the analysis of molecular dynamics simulations.
<em>Journal of Computational Chemistry</em>, 32(10):2319–2327, July 2011.
00778.
URL: <a href="http://doi.wiley.com/10.1002/jcc.21787">http://doi.wiley.com/10.1002/jcc.21787</a>, <a href="https://doi.org/10.1002/jcc.21787">doi:10.1002/jcc.21787</a>.

[5] Hai Nguyen, David&nbsp;A Case, and Alexander&nbsp;S Rose.
<span class="bibtex-protected">NGLview</span>–interactive molecular graphics for <span class="bibtex-protected">Jupyter</span> notebooks.
<em>Bioinformatics</em>, 34(7):1241–1242, April 2018.
00024.
URL: <a href="https://academic.oup.com/bioinformatics/article/34/7/1241/4721781">https://academic.oup.com/bioinformatics/article/34/7/1241/4721781</a>, <a href="https://doi.org/10.1093/bioinformatics/btx789">doi:10.1093/bioinformatics/btx789</a>.

[6] Douglas&nbsp;L. Theobald.
Rapid calculation of <span class="bibtex-protected">RMSDs</span> using a quaternion-based characteristic polynomial.
<em>Acta Crystallographica Section A Foundations of Crystallography</em>, 61(4):478–480, July 2005.
00127.
URL: <a href="http://scripts.iucr.org/cgi-bin/paper?S0108767305015266">http://scripts.iucr.org/cgi-bin/paper?S0108767305015266</a>, <a href="https://doi.org/10.1107/S0108767305015266">doi:10.1107/S0108767305015266</a>.